In [2]:
%matplotlib inline 

import os
import glob
import numpy as np
import pandas as pd
from skimage import io 
import rasterio
import subprocess as sp
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
mu_sd= []
sca_km= []
p_basin = []

In [ ]:
path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/'
#path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/clipped/20170401_clipped.tif'


##LOOP - change path to f....2 spots)
for f in sorted(glob.glob(os.path.join(path, '*.tif'))):
    print(f)
    raster = io.imread(f) #path
    # print("raster shape",raster.shape)

    # io.imshow(raster[:,:,0]) #snow depth, band1
    # io.show()

    # io.imshow(raster[:,:,1]) #alpha mask, band2
    # io.show()


    # convert to numpy array
    array = np.array(raster[:,:,0]) #RAW array of lidar tif
    # print("array shape",array.shape)

    #Ernesto method....
    # a=array.copy()
    sno_mask = array.copy()
    
    sno_mask[sno_mask > 0] = 1 #make SCA mask

    #sno_mask is matrix of 0 (no snow) and 1's (snow)
#     plt.imshow(sno_mask,alpha=None)
#     plt.title('Mask Layer (1=snow cover)')
#     plt.show()

    # meanSD -- caclulate mean snow depth (meanSD), without zero snow pixels
    mu_sd = np.mean(sno_mask) #mean snow depth, from sca 
    print("mean snow depth", mu_sd,'meters')


    # SCA -- pixs snow cover * pixel size
    num_pix = np.sum(sno_mask) #number of pixels with snow cover (sum the 1's)
#     print("number pixels with snow cover",num_pix)
    
    ras =  rasterio.open(f) #CHANGE FOR LOOP
    gt = ras.transform
    pixelSizeX = gt[0]
    pixelSizeY =-gt[4] #notice the -gt
#     print('pixel size (x,y) = ',pixelSizeX, pixelSizeY)

    sca = num_pix * pixelSizeX * pixelSizeY
    sca_km = sca/1000**2
    p_basin = sca_km/1180 *100  #per Hedrick et al. 2018
    print('Snow Covered Area (SCA) = ', sca, 'm^2')
    print('Snow Covered Area (SCA) = ', sca_km, 'km^2')
    print('% Basin snow cover', p_basin, '%')
    
#     array.close()
#     sno_mask.close()
    ras.close()

In [4]:
mu_sd= []
sca_km= []
p_basin = []

col=['a','b','c']

for i in range(5):
   
    mu_sd.append(100*i)
    sca_km.append(5-i)
    p_basin.append(i)
#     mu_= 100*i
#     print(mu_)
#     sca_km= 4+6+i
#     p_basin = .5*i

print(mu_sd)
#     mu_sd.append(md_) 
#     sca_km.append() 
#     p_basin.append()

df1 = pd.DataFrame.from_items(mu_sd)
# pd.DataFrame.from_items(sca_km)
# pd.DataFrame.from_items(p_basin)
# pd.DataFrame(np.array([[mu_sd, sca_km, p_basin]]),columns=['meanSD (m)','SCA (km)','% basin'])

# mu_sd=[5,4,3]
# sca_km=[6,7,8]
# p_basin = [1,2,3]

# pd.DataFrame.append(np.array([[mu_sd, sca_km, p_basin]]),columns=['meanSD','SCA (km)','% basin'])


[0, 100, 200, 300, 400]


/home/meganmason/Documents/projects/thesis/analysis/virtual_envs/data_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.


TypeError: zip argument #1 must support iteration